In [3]:
# evalPoly()

def evalPoly(a, x):
  n = len(a) - 1 
  p = a[n]
  dp = 0.0 + 0.0j
  ddp = 0.0 + 0.0j
  for i in range (1, n+1):
    ddp = ddp * x + 2 * dp
    dp = p + x * dp
    p = a[n-i] + x * p
  return p, dp, ddp

In [4]:
# Laguerre법
# 내가 직접 해봐야지 어쩔 수 잇나
# 크롬이 코딩용으로는 전체화면도 되고 괜찮은듯/ 평소엔 사파리, 코딩은 크롬으로만 
import cmath
import numpy as np
from random import random

def PolyRoots(a, tol=1.0E-12):
  def laguerre(a, tol):
    x = random()    # 시작값
    n = len(a) - 1    # 차수
    for i in range(30):
      p, dp, ddp = evalPoly(a, x)
      if abs(p) < tol :   
        return x
      g = dp / p
      h = g**2 - ddp / p
      f = cmath.sqrt((n-1)*(n*h - g**2))
      if abs(g + f) > abs(g - f):
        dx = n/(g + f)
      else : 
        dx = n/(g - f)
      x = x - dx    # 좌변 x는 개션된 근사값 r/ 우변 x는 random 시작해
      if abs(dx) < tol :
        return x
    print("계산횟수 초과")

  def deflPoly(a, root):    # deflection 축약
    n = len(a) - 1
    b = [(0.0 + 0.0j)] * n
    b[n-1] = a[n]
    for i in range(n-2, -1, -1):
      b[i] = a[i+1] + root * b[i+1]
    return b

  n = len(a) - 1
  roots = np.zeros((n), dtype=complex)
  for i in range(n):
    x = laguerre(a, tol)    # 차수만큼 근사해 찾는 과정 반복
    if abs(x.imag) < tol :    # x의 복소수 부분 이 허용오차보다 작으면 x의 실수를 입력
      x = x.real
    roots[i] = x    # root(0 array)의 i번째 행이 x 참조
    a = deflPoly(a, x)    # 축약한 상수열 b를 다시 a로 (차수 낮아진 방정식의 상수열)
  return roots    # 차수만큼 반복 다 하면 roots array에 해들이 나열됨
  


In [5]:
# 풀어보자 

import numpy as np

a = np.array([-250.0, 155.0, -9.0, -5.0, 1.0])    # 다항식의 상수 넣음/ 자동으로 다항식 만들고
x = PolyRoots(a)
print('x =', x)   # 해를 출력

x = [ 2.+0.j  4.-3.j  4.+3.j -5.+0.j]


In [8]:
## swapRC 모듈
''' vc = swapRows(vm,i,j).
        벡터 또는 행렬 [vm]의 i행과 j행을 교환
    vc = swapCols(vm,i,j).
        행렬 [vm]의 i열과 j열을 교환
'''
def swapRows(vm,i,j):
    vc = vm.copy()
    if len(vc.shape) == 1:
        t = vc[i]
        vc[i] = vc[j]
        vc[j] = t
    else:
        vc[[i,j],:] = vc[[j,i],:]
    return vc

def swapCols(vm,i,j):
    vc = vm.copy()
    vc[:,[i,j]] = vc[:,[j,i]]
    return vc

In [10]:
# GaussPivot.py
# 피봇팅을 이용한 Gauss 소거법
''' x = GaussPivot(a,b,tol=1.0e-12).
    연립선형방정식 [a]{x} = {b}을
    행피봇팅을 하고 가우스 소거법으로 푼다.
'''
import sys
import numpy as np

def GaussPivot(a,b,tol=1.0e-12):
    n = len(b)
    
    # 크기 척도 설정
    s = np.zeros(n)
    for i in range(n):
        s[i] = max(np.abs(a[i,:]))
            
    for k in range(0,n-1):
        
        # 필요하면 행 교환
        p = np.argmax(np.abs(a[k:n,k])/s[k:n]) + k
        if abs(a[p,k]) < tol:
            print('행렬이 특이임')
            sys.exit()
        if p != k:
            b = SwapRC.swapRows(b,k,p)
            s = SwapRC.swapRows(s,k,p)
            a = SwapRC.swapRows(a,k,p)
            
        # 소거
        for i in range(k+1,n):
            if a[i,k] != 0.0:
                lam = a[i,k]/a[k,k]
                a[i,k+1:n] = a[i,k+1:n] - lam*a[k,k+1:n]
                b[i] = b[i] - lam*b[k]
    if abs(a[n-1,n-1]) < tol:
        print('행렬이 특이임')
        sys.exit()
                   
    # 후방 대입
    b[n-1] = b[n-1]/a[n-1,n-1]
    for k in range(n-2,-1,-1):
        b[k] = (b[k] - np.dot(a[k,k+1:n],b[k+1:n]))/a[k,k]
    return b


In [24]:
import numpy as np
import math
def NewtonRaphson(fn, x, tol=1.0E-9):
  for i in range(30):
    jac, f0 = jacobian(fn, x)
    if math.sqrt(np.dot(f0, f0) / len(x)) < tol :
      return x
    dx = GaussPivot(jac, -f0)   # J(x) * dx = -f(x)
    x = x + dx
    if math.sqrt(np.dot(dx, dx)) < tol * max(max(abs(x)), 1.0):    # x 절댓값의 max와 1 중 더 큰 값
      return x, i   # 해와 반복횟수 출력
  print('반복횟수 초과')    # 30번 지나면 

def jacobian(fn, x):
  h = 1.0E-4
  n = len(x)
  jac = np.zeros((n, n))    # n*n 행렬 
  f0 = fn(x)    # x 방정식 초기를 f0? 
  for i in range(30):
    temp = x[i]
    x[i] = temp + h
    f1 = fn(x)
    x[i] = temp
    jac[:, i] = (f1 - f0)/h   # 모든행, i열 
    return jac, f0

In [26]:
# 집중해서 해보자 
import math
import numpy as np

def f(x):
  f = np.zeros(len(x))    # x의 크기만큼 0 배열
  f[0] = math.sin(x[0]) + x[1]**2 + math.log(x[2]) - 7.0
  f[1] = 3.0*x[0] + 2**x[1] - x[2]**3 + 1.0
  f[2] = x[0] + x[1] + x[2] - 5.0 
  return f

if __name__ == '__main__' :
  x = np.array([1.0, 1.0, 1.0])

  x, iter = NewtonRaphson(f, x)
  print('x = ', x)
  print('반복횟수 =', iter)

행렬이 특이임


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
